In [2]:
from pyomo.environ import *
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# ====================================================================================================
#   Model Control
# ====================================================================================================
# Length of one time step
time_interval = 3600 # sec
# Number of time steps in one run
time_steps = 24 

# ====================================================================================================
#   PV Design
# ====================================================================================================
# The width of the solar panel row/array
# [The edge that rotates about its midpoint with a changing tilt]
width_p = 2
# The length of the solar panel row/array
length_p = 20
# The height of the surface at its centerline
height_p = 2
# Site azimuth [north-south rows: 0 deg; east-west rows: 90 deg]
site_azimuth = 0
# Interrow spacing: the distance between the center of the panels of nearby rows
interrow_spacing_p = 6
# Intrarow spacing: the distance between the center of the nearby panels in one row
intrarow_spacing = 0
# row number: Number of rows defined in inputs
row_number = 14
# Panel number in a row
surf_number = 1
# Domain border length: The distance from the edge of the horizontal panels to the domain boundary
border_do = 2
# Maximum tracking angle
max_angle = 60
# Maximum change in tilt angle in one time step
delta_theta = 10
# Backtracking
backtrack = True

# ====================================================================================================
#   Weather
# ====================================================================================================
# PAR to total spectrum ratio [generally 0.45 - 0.53]
ratio_PAR = 0.5
# PAR photon energy
qc_PAR = 4.6 # umol/J
# DLI target
DLI_target = 25 # mol/m2/day

# ====================================================================================================
#   Power Generation
# ====================================================================================================
# Power price
power_price = 50 # $/MWh
# Number of modules does not matter for this calculation unless including the size of site
# Nameplate DC rating [generation at 1000 W/m2]
pdc0 = 230 
# Temperature coefficient [1/C]
gamma_pdc = -0.003
# Reference temperature [C]
temperature_ref = 25

In [22]:
# ====================================================================================================
#   Input Data
# ====================================================================================================
# Solar radiation data (New York in summer) for 24 hours
DNI = np.array([0, 0, 0, 0, 0, 36, 87, 271, 669, 412, 225, 68, 13, 26, 27, 2, 197, 0, 0, 0, 0, 0, 0, 0])  # W/m²
DHI = np.array([0, 0, 0, 0, 0, 55, 131, 205, 154, 319, 433, 434, 265, 343, 314, 167, 234, 38, 16, 1, 0, 0, 0, 0])  # W/m²
phi_a = np.array([350.86, 5.92, 20.6, 34.12, 46.2, 56.95, 66.7, 75.94, 85.25, 95.45, 108.02, 126.06, 155.48, 196.12, 228.73, 248.67, 262.09, 272.65, 282.07, 291.26, 300.87, 311.36, 323.12, 336.34])
theta_e = np.array([-23.71, -24.02, -21.47, -16.37, -9.23, 0.01, 9.29, 19.71, 30.6, 41.64, 52.46, 62.33, 69.43, 70.28, 64.23, 54.74, 44.05, 33.02, 22.07, 11.52, 1.83, -7.43, -14.94, -20.53])

temperature = None

In [17]:
# ====================================================================================================
#   Parameter Initialization
# ====================================================================================================
# Initial tracking angle
initial_theta = 0

# Relative azimuth angle
phi_r = phi_a - site_azimuth

# Trignometry
sin_phi_r = np.sin(np.deg2rad(phi_r))
cos_phi_r = np.cos(np.deg2rad(phi_r))
tan_theta_e = np.tan(np.deg2rad(theta_e))

# # Solar elevation angle on the xz-plane (deg)
# # [positive means positive x direction]
# theta_e_xz = np.array(np.rad2deg([
#     np.arctan(np.sin(theta_e[t]) / (np.cos(theta_e[t]) * np.sin(phi_r[t]))) 
#     for t in range(time_steps)
# ]))

# theta_e_xz_raw = np.array(np.rad2deg([
#     np.arctan(np.sin(theta_e[t]) / (np.cos(theta_e[t]) * np.abs(np.sin(phi_r[t])))) 
#     for t in range(time_steps)
# ]))

# # Optimal tilt angle for maximum power production (deg)
# optimal_angle = np.array(np.rad2deg([
#     np.pi / 2 * np.sin(phi_r[t]) / np.abs(np.sin(phi_r[t])) - 
#     np.deg2rad(theta_e_xz)
#     for t in range(time_steps)
# ]))

# Remove values when sun is below the horizon
DHI[theta_e<0] = 0.0
DNI[theta_e<0] = 0.0
# GHI[theta_e<0] = 0.0 # generally not needed since it can be calculated based on DHI and DNI

# Ground cover ratio
gcr = width_p / interrow_spacing_p

# ****************************************************************************************************
#   Domain y direction is defined by the panel rows [not real north-south]
#   Tilt angles are defined using the right-hand rule
#   Distances computed from the domain left and domain bottom
# ****************************************************************************************************
#   Domain example:                 ||          []          []          []          []          ||
#                                   ||          []          []          []          []          ||
#   → x-direction                   ||          []          []          []          []          ||
#   ↑ y-direction                   ||          []          []          []          []          ||
#   * z-direction                   ||          []          []          []          []          ||
#                                   ||          []          []          []          []          ||
#                                   ||          []          []          []          []          ||
#                                   ||          []          []          []          []          ||
#                                   ||          []          []          []          []          ||
#                                   ||          []          []          []          []          ||
#   ||: Domain boundary             ||          []          []          []          []          ||
#   []: Solar panels                ||          []          []          []          []          ||
# ****************************************************************************************************

In [ ]:
# ====================================================================================================
#   Create Pyomo model
# ====================================================================================================
model = ConcreteModel()

# ====================================================================================================
#   Decision variable
# ====================================================================================================
# Tilt angle θ, bounded between -60 and 60 degrees (converted to radians)
model.θ = Var(range(time_steps), 
              bounds=(-max_angle / 180 * np.pi, max_angle / 180 * np.pi), 
              initialize=initial_theta)

# ====================================================================================================
#   Constraint
# ====================================================================================================
# ----------------------------------------------------------------------------------------------------
#   DLI
# ----------------------------------------------------------------------------------------------------
# Shadow length: Projected panel width on the ground 
model.panel_width_shadow = Var(range(time_steps), within=NonNegativeReals)
model.shadow_constraints = ConstraintList()

# Shadow length [Option 1: Simiplified ART]
for t in range(time_steps):
    model.shadow_constraints.add(
        model.panel_width_shadow[t] >= sin_phi_r[t] / tan_theta_e[t] * 2 * sin(model.θ[t]) + 2 * cos(model.θ[t])
    )
    model.shadow_constraints.add(
        model.panel_width_shadow[t] >= - sin_phi_r[t] / tan_theta_e[t] * 2 * sin(model.θ[t]) + 2 * cos(model.θ[t])
    )

# Shadow length [Option 2: Shadow projection]
# for t in range(time_steps):
#     model.shadow_constraints.add(model.shadow_length[t] >= cos(model.θ[t] - optimal_angle[t]) * width_p / np.sin(theta_e_xz_raw[t]))
#     model.shadow_constraints.add(model.shadow_length[t] >= -cos(model.θ[t] - optimal_angle[t]) * width_p / np.sin(theta_e_xz_raw[t]))

# DLI expression
# Important: gcr here is used as an approximation for sky view factors. However, svf can reach 0.62 when tilt is 90 deg with a gcr of 0.5.
#            When gcr is smaller than 0.5, this difference is negligible, and gcr greater than 0.5 is usually impossible in applications.
def DLI_expr(model, t):
    return ((interrow_spacing_p - model.shadow_length[t]) * (sin(theta_e[t]) * DNI[t] + DHI[t] * gcr) + 
            model.shadow_length[t] * DNI[t] * gcr) / interrow_spacing_p * ratio_PAR * qc_PAR * 24 * 3600 / 1e6
model.DLI = Expression(range(time_steps), rule=DLI_expr)

# DLI constraint
model.DLI_constraint = Constraint(expr=sum(model.DLI[t] for t in range(time_steps)) / time_steps >= DLI_target)

# May add a temperature effect later

# ----------------------------------------------------------------------------------------------------
#   Backtracking
# ----------------------------------------------------------------------------------------------------
if backtrack:
    def shadow_length_constraint_rule(model, t):
        return model.shadow_length[t] <= interrow_spacing_p

    model.shadow_length_constraint = Constraint(model.T, rule=shadow_length_constraint_rule)

# ----------------------------------------------------------------------------------------------------
#   Rate of change in tilt angle
# ----------------------------------------------------------------------------------------------------
# Constraints for limiting rate of change of tilt angle
if delta_theta != None:
    model.angle_change_constraints = ConstraintList()
    for t in range(time_steps - 1):
        model.angle_change_constraints.add(model.θ[t + 1] - model.θ[t] <= delta_theta / 180 * np.pi)
        model.angle_change_constraints.add(model.θ[t + 1] - model.θ[t] >= - delta_theta / 180 * np.pi)

# ====================================================================================================
#   Objective: Power Generation
# ====================================================================================================
# AOI expression
def cos_aoi_expr(model, t):
    return cos(model.θ[t]) * sin(theta_e[t]) + sin(model.θ[t]) * cos(theta_e[t]) * sin(phi_r[t])

model.cos_aoi = Expression(range(time_steps), rule=cos_aoi_expr)

# Power generation expression
# Important: Diffuse radiation is modified by (cos(θ) + 1) / 2 for mono-facial panels
#            This can be inaccurate since diffuse radiation is not zero when tilt is 90 deg
# width_p * length_p not needed since we only maximize power generation which is unrelated to these
def power_generation_expr(model, t):
    return  pdc0 * (model.cos_aoi[t] * DNI[t] + DHI[t] * (cos(model.θ[t]) + 1) / 2) / 1000 
    # * (1 + gamma_pdc * (temperature - temperature_ref))
# Simple temperature model may be added

model.power_generation = Expression(range(time_steps), rule=power_generation_expr)

# Objective function: maximize power generation
# power_price not needed unless some panelties or DLI panalities apply
def objective_rule(model):
    return sum(model.power_generation[t] for t in range(time_steps)) / 1e6 * power_price

model.objective = Objective(rule=objective_rule, sense=maximize)

# ====================================================================================================
#   Solution
# ====================================================================================================
# Solve the optimization problem using Ipopt
solver = SolverFactory('ipopt', executable='C:/IPOPT/Ipopt-3.14.16-win64-msvs2019-md/bin/ipopt.exe')
solver.solve(model, tee=False)